# Import Libraries

In [1]:
import time
from array import array
import os
import shutil
import time
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
from glob import glob
import gc
import warnings
warnings.filterwarnings('ignore')

c:\Users\mdk05\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get Data Path

In [2]:
data_path = "G:/Ubiosis/recon"

subject_list = list(os.listdir(data_path))

# Get X

In [9]:
def dat_process(dat_path):
    # dat file open by lines
    with open(dat_path, encoding='euc-kr') as dat_data:
        data = dat_data.read().splitlines()
    
    if len(data) > 35: # 정상적으로 관측되었을 경우
        # Meta 파일 추출 (FIRST TEST COUNT, RISER RADIUS)
        ftc_row = data[10]
        ftc_row = ftc_row.split(",")
        ftc = ftc_row[5]

        rr_row = data[24]
        rr_row = rr_row.split(",")
        rr = rr_row[5]
        
        meta_dict = {
            'FIRST_TEST_COUNT':ftc,
            'RISER RADIUS':rr
        }
        meta_df = pd.DataFrame(meta_dict, index=[0])
        
        # Column 정의
        column = ["Time", "Temp(F)", "CIS_1", "CIS_2"]

        # 관측값 추출
        values_list = []
        # line별 data 추출
        for idx in range(32,len(data)-2):
            tmp_list=[]
            value_list = data[idx] # raw 데이터 담는 리스트
            value_list = value_list.split(",")[:-1] # line에서 , 기준으로 데이터 분리

            values = [] # 정리한 데이터를 담는 리스트
            for v in value_list:
                value = v.strip() # 데이터 앞/뒤의 공백 문자 제거
                values.append(value)

            tmp_list.append(values[0])
            tmp_list.append(values[1])
            tmp_list.append(values[2])
            tmp_list.append(values[3])
            values_list.append(tmp_list)

        value_df = pd.DataFrame(columns=column,data=values_list)

    return meta_df, value_df

In [10]:
trg_path = "G:/Ubiosis/data"
if os.path.isdir(trg_path) == False:
    os.mkdir(trg_path)

err_list = []
for idx in trange(len(subject_list)):
    # 대상자 ID 정보 호출 후 경로 변수 생성
    subject = subject_list[idx]
    subject_path = os.path.join(data_path, subject)
    
    # Target 폴더에 ID 폴더 생성
    new_id_path = os.path.join(trg_path, subject)
    if os.path.isdir(new_id_path) == False:
        os.mkdir(new_id_path)
    
    org_file_list = list(os.listdir(subject_path))
    for file in org_file_list:
        file_ext = file.split(".")[-1]
        
        if file_ext == "dat":
            meta_df, value_df = dat_process(os.path.join(subject_path, file))
            meta_path = os.path.join(new_id_path, (subject + "_meta.csv"))
            value_path = os.path.join(new_id_path, (subject + "_x.csv"))
            meta_df.to_csv(meta_path, index=False)
            value_df.to_csv(value_path, index=False)

100%|██████████| 81430/81430 [1:17:01<00:00, 17.62it/s]
